In [1]:
from bs4 import BeautifulSoup
from requests import get

In [2]:
banks_page = get('https://tockanai.hr/banke/')

soup = BeautifulSoup(banks_page.content, 'html.parser')

In [3]:
div = soup.find_all("div", attrs={"class": "vc_tta-panels-container"})

In [19]:
banks = {}

for item in div:

    bank = str(item.find_next('h3'))
    
    bank_name = bank[4:-5].strip()

    p = str(item.findChildren('p'))

    try:
        s = p.index('Poštanski broj i grad')
        e = p.index('<br/>', s+len('Poštanski broj i grad'))

        res = p[s+len('Poštanski broj i grad:')+7:e]

        if ', ' in p[s+len('Poštanski broj i grad:')+7:e]:   
            res = res.replace(',', '') 

        city = res.strip()

    except ValueError:
        city = ''


    try:
        s = p.index('Adresa')
        e = p.index('<br/>', s+7)

        address = p[s+7:e].strip()
        
    except ValueError:
        address = ''

    try:
        s = p.index('OIB')
        e = p.index('<br/>', s+4)

        oib = p[s+4:e].strip()

    except ValueError:
        oib = ''
    
    try:
        s = p.index('Matični broj')
        e = p.index('<br/>', s+len('Matični broj'))

        mib = p[s+len('Matični broj:'):e].strip()
  
    except ValueError:
        mib = ''
    
    banks[bank_name] = {'oib': oib, 'mib': mib, 'address': address, 'city': city}
    
    

In [35]:
from pandas import DataFrame

In [38]:
df = DataFrame.from_dict(banks)
df = df.T

In [37]:
df

,ADDIKO BANK d.d.,AGRAM BANKA d.d. Zagreb,BANKA KOVANICA d.d.,BKS BANK AG,CROATIA BANKA d.d.,ERSTE &amp; STEIERMÄRKISCHE BANK d.d.,HRVATSKA POŠTANSKA BANKA d.d. Zagreb,IMEX BANKA d.d. Split,ISTARSKA KREDITNA BANKA UMAG d.d. Umag,JADRANSKA BANKA d.d. Šibenik,...,PRIVREDNA BANKA ZAGREB d.d. Zagreb,RAIFFEISENBANK AUSTRIA d.d. Zagreb,SAMOBORSKA BANKA d.d. Samobor,SBERBANK d.d. Zagreb,SLATINSKA BANKA d.d. Slatina,SPLITSKA BANKA d.d. Split (OTP Grupa),ŠTEDBANKA d.d. u likvidaciji,TESLA ŠTEDNA BANKA d.d. Zagreb u stečaju,VENETO BANKA d.d. Zagreb,ZAGREBAČKA BANKA d.d. Zagreb
oib,14036333877,70663193635,33039197637,02138784111,32247795989,23057039320,87939104217,99326633206,65723536010,02899494784,...,02535697732,53056966535,13806526186,78427478595,42252496579,69326397242,58063088591,82901735413,81712716992,92963223473
mib,1198947,80003981,1326287,40364313,03467988,3337367,3777928,0971359,3463958,3019349,...,3269841,80002366,3113680,1260405,3999092,60000488,813737,02500256,3917185,3234495
address,Slavonska avenija 6,Ulica grada Vukovara 74,Petra Preradovića 29,Mljekarski trg 3,Roberta Frangeša Mihanovića 9,Jadranski trg 3A,Jurišićeva 4,Tolstojeva 6,Ernesta Miloša 1,Ante Starčevića 4,...,Radnička cesta 50,Magazinska cesta 69,Trg kralja Tomislava 8,Varšavska 9,V.Nazora 2,Ulica Domovinskog rata 61,Slavonska avenija 3,Trg J. F. Kennedyja 6 B,Draškovićeva 58a,Trg bana Josipa Jelačića 10
city,Zagreb,Zagreb,Varaždin,Rijeka,Zagreb,Rijeka,Zagreb,Split,Umag,Šibenik,...,Zagreb,Zagreb,Samobor,Zagreb,Slatina,Split,Zagreb,Zagreb,Zagreb,Zagreb


In [21]:
import json

In [34]:
j = '['

for bank in banks:

    j += f'{{"model": "third_parties_app.bank","pk":"{banks[bank]["oib"]}", "fields": {"mib":"{banks[bank]["mib"]}","bank_name":"{bank}","address":"{banks[bank]["address"]}","city":{banks[bank]["city"]}}}},'

j = j[:-1] + ']'

ValueError: Invalid format specifier